In [25]:
import pandas as pd

In [15]:
url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
url_white = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

In [16]:
# read csv

df_red = pd.read_csv(url_red, sep=";")
df_white = pd.read_csv(url_white, sep=";")

df_red["wine_type"] = "red"
df_white["wine_type"] = "white"

df_wine = pd.concat([df_red, df_white], ignore_index=True)

In [24]:
print(df_wine.head())
print(df_wine.describe())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality wine_type  quality_label  
0      9.4        5       red          

In [19]:
df_wine["quality_label"]= (df_wine["quality"] >=7).astype(int)
df_wine[["quality" , "quality_label" , "wine_type"]].head()

,quality,quality_label,wine_type
0,5,0,red
1,5,0,red
2,5,0,red
3,6,0,red
4,5,0,red


In [20]:
x = df_wine.drop(["quality" , "quality_label", "wine_type"] , axis=1)
y = df_wine["quality_label"]

In [21]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(x ,y , test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import LogisticRegression


In [23]:
model = LogisticRegression(max_iter =1000)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [26]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90      1048
           1       0.65      0.25      0.36       252

    accuracy                           0.83      1300
   macro avg       0.75      0.61      0.63      1300
weighted avg       0.81      0.83      0.80      1300



In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# model oluşturuyoruz
model_linear = LinearRegression()

# eğitim
model_linear.fit(X_train , y_train)

# tahmin
y_pred_linear = model_linear.predict(X_test)

# Hata hesaplama
mse = mean_squared_error(y_test , y_pred_linear)
rmse = mse ** 0.5

print("MSE : " , mse)
print("RMSE:" , rmse)

MSE :  0.126852637084793
RMSE: 0.3561637784570365


In [28]:
from sklearn.tree import DecisionTreeClassifier

# karar ağacı model oluşturmak

dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X_train , y_train)

y_pred_dt = dt_model.predict(X_test)

print("Decision Sonuçları: ")
print(classification_report(y_test , y_pred_dt))

Decision Sonuçları: 
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      1048
           1       0.57      0.63      0.60       252

    accuracy                           0.83      1300
   macro avg       0.74      0.76      0.75      1300
weighted avg       0.84      0.83      0.84      1300



In [29]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train , y_train)

y_pred_rf = rf_model.predict(X_test)

print("Random Forest sonuçları : ")
print(classification_report(y_test , y_pred_rf))

Random Forest sonuçları : 
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1048
           1       0.79      0.57      0.66       252

    accuracy                           0.89      1300
   macro avg       0.84      0.77      0.80      1300
weighted avg       0.88      0.89      0.88      1300



0 Sınıfında yüksek bir precision var . Model 0 sınıfına karşı doğru tahminlerde bulunuyor..

1 sınıfında ise precision düşük %21 lik kısım 0 sınıfına ait .
--Model 1 tahminlerinde yanılma payı taşıyor

In [30]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(random_state=42)

xgb_model.fit(X_train , y_train)

y_pred_xgb = xgb_model.predict(X_test)


print("XGBoost sonuçları : ")
print(classification_report(y_test , y_pred_xgb))

XGBoost sonuçları : 
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1048
           1       0.74      0.61      0.67       252

    accuracy                           0.88      1300
   macro avg       0.83      0.78      0.80      1300
weighted avg       0.88      0.88      0.88      1300



1 sınıfına daha duyarlı olmasını sağlayacağız.


# Class Weights Kullanımı

In [31]:
# scale_pos_weight azınlık sınıfa daha çok ağırlık verir.

from xgboost import XGBClassifier

scale_pos_weight = len(y_train) / (2 * sum(y_train == 1 ))  # hesaplama kısmı

# sınıf dengesizliğini dikkate almasını sağlamak
model = XGBClassifier(scale_pos_weight=scale_pos_weight)

model.fit(X_train , y_train)

# tahmin ve değerlendirme
y_pred = model.predict(X_test)

print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1048
           1       0.70      0.68      0.69       252

    accuracy                           0.88      1300
   macro avg       0.81      0.80      0.81      1300
weighted avg       0.88      0.88      0.88      1300



In [32]:
null_values = df_wine.isnull().sum()

# Null değer sayısını görüntüleme
print(null_values)

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
quality_label           0
dtype: int64


In [33]:
from sklearn.preprocessing import StandardScaler

# Sayısal özellikleri belirleyelim
# Use the original DataFrame for selecting numerical features
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns

# Sayısal verileri ölçeklendirelim
scaler = StandardScaler()

# Fit and transform on X_train and X_test separately
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# Check the scaled data (optional)
print(X_train.head())

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1916      -0.468229         -0.599512     0.205467        0.464612  -0.712801   
947        0.851500         -0.352849     1.104063       -0.702999   1.067470   
877        0.385713          2.329612    -2.144710       -0.702999   0.234762   
2927      -1.555065          0.202143     0.067221       -0.890650  -0.798944   
6063      -0.468229         -0.476180     1.657046        2.070076  -0.081092   

      free sulfur dioxide  total sulfur dioxide   density        pH  \
1916             0.311659              0.343566 -0.296146 -0.168884   
947             -1.408735             -1.847518 -0.203072  0.269679   
877              0.024927             -1.295294 -0.326062  1.209455   
2927            -1.007309             -0.618374 -1.492810  0.958848   
6063             0.082273              0.450448  1.133204 -0.670098   

      sulphates   alcohol  
1916  -1.083266  0.013766  
947    0.598577  1.608717  
87

In [34]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Özellikleri ve hedefi ayır
X = df_wine.drop(["quality", "quality_label"], axis=1)
X = pd.get_dummies(X, drop_first=True)  # Kategorik değişkenleri dummies'e çevir
y = df_wine["quality_label"]

# Veriyi ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Pipeline ve parametreler
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(solver='liblinear'))
])

param_grid = {
    'clf__C': [0.1, 1, 10],
    'clf__class_weight': [
        {0: 1, 1: 1},
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        'balanced'
    ]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)

# Test sonucu
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))


Best Parameters: {'clf__C': 10, 'clf__class_weight': {0: 1, 1: 2}}
              precision    recall  f1-score   support

           0       0.88      0.86      0.87      1044
           1       0.49      0.54      0.51       256

    accuracy                           0.80      1300
   macro avg       0.69      0.70      0.69      1300
weighted avg       0.81      0.80      0.80      1300



In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Pipeline tanımı
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

# Grid parametreleri
param_grid_rf = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20],
    'clf__class_weight': [None, {0:1, 1:2}, 'balanced']
}

# GridSearchCV tanımı
grid_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=5, scoring='f1_macro', n_jobs=-1)
grid_rf.fit(X_train, y_train)

# En iyi parametreler ve test sonucu
print("Best Parameters:", grid_rf.best_params_)

y_pred_rf = grid_rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))


Best Parameters: {'clf__class_weight': None, 'clf__max_depth': None, 'clf__n_estimators': 200}
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1044
           1       0.80      0.57      0.66       256

    accuracy                           0.89      1300
   macro avg       0.85      0.77      0.80      1300
weighted avg       0.88      0.89      0.88      1300

